<a href="https://colab.research.google.com/github/argxsL/CCMACLRL_EXERCISES_COM231ML/blob/main/Exercise3/Exercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 3

Use logistic regression to predict the probability of a passenger surviving.

https://www.kaggle.com/competitions/titanic/overview

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression

## Dataset File

In [ ]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES_OLD/blob/f5fdfba1aabbbafcf2fe49ca6e1c8f92f69d1458/datasets/titanic/train.csv?raw=true'
df = pd.read_csv(dataset_url)

## Test File

In [ ]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES_OLD/blob/f5fdfba1aabbbafcf2fe49ca6e1c8f92f69d1458/datasets/titanic/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [ ]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


## Experiments

### Model 1

In [160]:
md1 = df.copy()
md_1 = dt.copy()

In [161]:
## new feature combine siblings and parents
md1['FamilySize'] = md1['SibSp'] + md1['Parch'] + 1
md1=md1.drop(columns=['SibSp', 'Parch'])

md_1['FamilySize'] = md_1['SibSp'] + md_1['Parch'] + 1
md_1=md_1.drop(columns=['SibSp', 'Parch'])

In [162]:
bins = [0, 5, 17, 25, 50, 80]
labels = ['Infant', 'Kid', 'Young', 'Adult', 'Old']
md1['Demographic'] = pd.cut(md1['Age'], bins=bins, labels=labels)
md_1['Demographic'] = pd.cut(md_1['Age'], bins=bins, labels=labels)

In [163]:
md1 = pd.get_dummies(md1, columns=['Sex','Embarked','Demographic'])
md_1 = pd.get_dummies(md_1, columns=['Sex','Embarked','Demographic'])

In [164]:
## delete not needed
md1 = md1.drop(columns=['Name','Ticket','Cabin','Age'])
md1.head()

md_1 = md_1.drop(columns=['Name','Ticket','Cabin','Age'])
md_1.head()

,PassengerId,Pclass,Fare,FamilySize,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Demographic_Infant,Demographic_Kid,Demographic_Young,Demographic_Adult,Demographic_Old
0,892,3,7.8292,1,False,True,False,True,False,False,False,False,True,False
1,893,3,7.0000,2,True,False,False,False,True,False,False,False,True,False
2,894,2,9.6875,1,False,True,False,True,False,False,False,False,False,True
3,895,3,8.6625,1,False,True,False,False,True,False,False,False,True,False
4,896,3,12.2875,3,True,False,False,False,True,False,False,True,False,False


In [152]:
md_1.head(153)

,Pclass,Fare,FamilySize,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Demographic_Infant,Demographic_Kid,Demographic_Young,Demographic_Adult,Demographic_Old
0,3,7.829200,1,False,True,False,True,False,False,False,False,True,False
1,3,7.000000,2,True,False,False,False,True,False,False,False,True,False
2,2,9.687500,1,False,True,False,True,False,False,False,False,False,True
3,3,8.662500,1,False,True,False,False,True,False,False,False,True,False
4,3,12.287500,3,True,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,26.550000,1,False,True,False,False,True,False,False,False,False,False
149,2,26.000000,3,False,True,False,False,True,False,False,False,True,False
150,1,83.158300,2,True,False,True,False,False,False,False,True,False,False
151,3,7.895800,1,False,True,True,False,False,False,False,False,False,False


In [165]:
from sklearn.impute import SimpleImputer

mean_imputer = SimpleImputer(strategy='mean')
md_1['Fare'] = mean_imputer.fit_transform(md_1[['Fare']])

In [166]:
X_train = md1.drop(columns=['Survived'])
y_train = md1['Survived']
X_test = md_1

In [192]:
model1 = LogisticRegression()
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)

model1.score(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.813692480359147

In [191]:
model1.score(X_test, y_pred)

1.0

## Sample Submission File

In [187]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES_OLD/blob/f5fdfba1aabbbafcf2fe49ca6e1c8f92f69d1458/datasets/titanic/gender_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [188]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


## Generate Results

In [189]:
id = sf.pop('PassengerId')
y_pred = model1.predict(md_1)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'PassengerID': id,
    'Survived': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv


In [ ]:
# Create a submission DataFrame
submission_df = pd.DataFrame({
    'PassengerId': dt['PassengerId'],
    'Survived': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")